In [16]:
import os
from dotenv import load_dotenv

load_dotenv() #.env dosyasini yüklüyor, API key'leri yüklüyor.

from langchain_openai import ChatOpenAI
from youtube_transcript_api import YouTubeTranscriptApi

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")


model = ChatOpenAI(model="gpt-4") #UniKassel hesabi gpt-4. Temperature eklenebilir, duruma göre belki evaluation asamasinda faydali olabilir.


In [26]:
#ChatGPT kodu bu: deneme icin bir telekom sayfasi getirmeye calistim ama istedgiim sayfanin icerigi görüntülenmiyor.

import requests
from bs4 import BeautifulSoup

url = 'https://www.telekom.de/hilfe/vertrag-rechnung/telekom-login/mobilfunk-verknuepfen'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Web sayfasındaki tüm paragrafları al
paragraphs = soup.find_all('p')
text_data = [p.get_text() for p in paragraphs]

# Tüm metni birleştir
full_text = ' '.join(text_data)
print(full_text)


Mobilfunk Dieser Warenkorb enthält {{ITEMS_COUNT}} Artikel. Zum Warenkorb Festnetz & TV Dieser Warenkorb enthält {{ITEMS_COUNT}} Artikel. Zum Warenkorb Magenta-WK Dieser Warenkorb enthält {{ITEMS_COUNT}} Artikel. Zum Warenkorb MeinMagenta App entdecken Zur App und Vorteile sichern! Kundencenter Im Kundencenter können Sie Verträge verwalten und auf Rechnungen, Verbrauchsanzeigen, Einstellungen und mehr zugreifen. Hilfe zum LoginNoch nicht registriert? Jetzt registrieren Login-Einstellungen bearbeiten   Hallo {{USER_NAME}} Sie möchten Ihren persönlichen Bereich verlassen?   So können Sie Ihren Mobilfunk-Vertrag mit einem Telekom Login verknüpfen: Melden Sie sich mit Ihrem Telekom Login im Kundencenter an und gehen Sie in der Kachel "Verträge" auf "Verträge verwalten". Klicken Sie dort auf "Vertrag hinzufügen". Nun wählen Sie zwischen "Privater Vertrag" oder "Geschäftlicher Vertrag". In der nächsten Ansicht sehen Sie die bereits verknüpften Verträge inklusive Nutzerrollen. Klicken Sie im 

In [17]:
# Deneme icin buraya bir tane Telekom Hilfe kanalindan bir video alyiorum, 
# komple youtube kanalini nasil transkript ettirip alabilirim buna bakmaliyim, 
# bir dünya video var cünkü.
# BURADA PROBLEM ALMANCA KARAKTERLERIN DÜZGÜN GÖSTERILMEMESI; ß,ü,ö,ä...

YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=BrsocJb-fAo&t=1525s&ab_channel=Underfitted"

transcript = YouTubeTranscriptApi.get_transcript("X3D008558Ek?si=YUbILctuzqEOHzOR", languages=['de', 'en'])

'''
# creating or overwriting a file "subtitles.txt" with 
# the info inside the context manager
with open("subtitles.txt", "w") as f:
   
        # iterating through each element of list srt
    for i in transcript:
        # writing each element of transcript on a new line
        f.write("{}\n".format(i))
'''

'\n# creating or overwriting a file "subtitles.txt" with \n# the info inside the context manager\nwith open("subtitles.txt", "w") as f:\n   \n        # iterating through each element of list srt\n    for i in transcript:\n        # writing each element of transcript on a new line\n        f.write("{}\n".format(i))\n'

In [18]:
# Burada OpenAI API request yapisi var, human, system vb...

from langchain_core.messages import HumanMessage, SystemMessage

'''
messages = [
    SystemMessage(content="Translate the following from Turkish into English"),
    HumanMessage(content="Bu benim ilk denemem!"),
]

result = model.invoke(messages)
'''


'\nmessages = [\n    SystemMessage(content="Translate the following from Turkish into English"),\n    HumanMessage(content="Bu benim ilk denemem!"),\n]\n\nresult = model.invoke(messages)\n'

# Video1: Overview - RAG From scratch

In [19]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents, Youtube
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM, yukarida zaten model diye gpt-4 tanimladik, temperature eklenebilir
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} #question: RunnablePassthrough kullanarak kullanıcıdan gelen soruyu işler.
    | prompt
    | model
    | StrOutputParser() 
    # StrOutputParser: Modelden gelen yanıtı ayrıştırır, karmasik olarak degilde direkt string olarak verir.
    # parser = StrOutputParser() chain = model | parser, bu sekilde de kullanilabilir.
)

# Question
rag_chain.invoke("What is Task Decomposition?")

/Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


"Task Decomposition is a process where complex tasks are broken down into smaller, simpler steps. This technique, often used in models to enhance performance on complicated tasks, allows for more efficient use of computation time during testing. It transforms a big task into multiple manageable tasks, providing insight into the model's thinking process."

## Video2: Indexing

In [20]:
# Documents
question = "My favorite pet is a cat."
document = "My favorite pet is a cat."

In [21]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

7

In [22]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)
# sorunun embedding olarak ne kadar uzun oldugunu yazdiriyor, 1536...

1536

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.9999999999999999


In [ ]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [ ]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000016EF5782210>


## Video3: Retrieval

In [ ]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
docs = retriever.invoke("What is Task Decomposition?")
len(docs)
print(docs)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

## Video4: Generation

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [ ]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [ ]:
# Chain
chain = prompt | llm

In [ ]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='Task Decomposition is a process where hard tasks are broken down into smaller and simpler steps. Techniques like Chain of Thought (CoT) and Tree of Thoughts are used to enhance model performance on complex tasks. CoT instructs the model to "think step by step" to utilize more test-time computation to decompose tasks, while Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure. Task decomposition can be done by an LLM with simple prompting, by using task-specific instructions, or with human inputs.', response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 331, 'total_tokens': 441}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8fa1913f-a4aa-46a0-90c9-a99604e60b41-0', usage_metadata={'input_tokens': 331, 'output_tokens': 110, 'total_tokens': 441})

In [ ]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [ ]:
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a process where hard tasks are broken down into smaller and simpler steps. This technique, often used to enhance model performance on complex tasks, transforms big tasks into multiple manageable tasks. It can be done by using simple prompting, task-specific instructions, or with human inputs.'